<a href="https://colab.research.google.com/github/Sivasubramaniansivanayagam/Recommendation-Model-ML/blob/main/Recommendation_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd

In [ ]:
# df=pd.read_csv("/content/drive/MyDrive/listenings.csv",encoding="unicode_escape")

In [ ]:
!pip install pyspark

     |████████████████████████████████| 281.4 MB 33 kB/s 
     |████████████████████████████████| 198 kB 58.1 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=237c4021ce10e4726ed9ece4170add56afcac32d80314bb2c38792902ada1dfd
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


In [ ]:
from pyspark.sql import SparkSession

In [ ]:
from pyspark.sql.functions import count,desc,col,max

In [ ]:

from pyspark.ml.feature import StringIndexer

In [ ]:
from pyspark.ml import Pipeline

In [ ]:
from pyspark.ml.recommendation import ALS # Ecommerce use


In [ ]:
from pyspark.ml.tuning import TrainValidationSplit,ParamGridBuilder

In [ ]:
spark=SparkSession.builder.appName("trial").getOrCreate()

In [ ]:
file_path="/content/drive/MyDrive/listenings.csv"

In [ ]:
df_listenings=spark.read.format("csv").option("header",True).option("inferSchema",True).load(file_path)

In [ ]:
df_listenings.show()

+-----------+-------------+--------------------+---------------+--------------------+
|    user_id|         date|               track|         artist|               album|
+-----------+-------------+--------------------+---------------+--------------------+
|000Silenced|1299680100000|           Price Tag|       Jessie J|         Who You Are|
|000Silenced|1299679920000|Price Tag (Acoust...|       Jessie J|           Price Tag|
|000Silenced|1299679440000|Be Mine! (Ballad ...|          Robyn|            Be Mine!|
|000Silenced|1299679200000|            Acapella|          Kelis|            Acapella|
|000Silenced|1299675660000|   I'm Not Invisible|      The Tease|   I'm Not Invisible|
|000Silenced|1297511400000|Bounce (Feat NORE...|       MSTRKRFT|         Fist of God|
|000Silenced|1294498440000|Don't Stop The Mu...|        Rihanna|Addicted 2 Bassli...|
|000Silenced|1292438340000|               ObZen|      Meshuggah|               ObZen|
|000Silenced|1292437740000|   Yama's Messengers|      

In [ ]:
number_of_rows=df_listenings.count()
number_of_columns=len(df_listenings.columns)

In [ ]:
print(number_of_rows)
print(number_of_columns)

14650594
5


In [ ]:
df_listenings=df_listenings.drop("date")

In [ ]:
df_listenings.show()

+-----------+--------------------+---------------+--------------------+
|    user_id|               track|         artist|               album|
+-----------+--------------------+---------------+--------------------+
|000Silenced|           Price Tag|       Jessie J|         Who You Are|
|000Silenced|Price Tag (Acoust...|       Jessie J|           Price Tag|
|000Silenced|Be Mine! (Ballad ...|          Robyn|            Be Mine!|
|000Silenced|            Acapella|          Kelis|            Acapella|
|000Silenced|   I'm Not Invisible|      The Tease|   I'm Not Invisible|
|000Silenced|Bounce (Feat NORE...|       MSTRKRFT|         Fist of God|
|000Silenced|Don't Stop The Mu...|        Rihanna|Addicted 2 Bassli...|
|000Silenced|               ObZen|      Meshuggah|               ObZen|
|000Silenced|   Yama's Messengers|         Gojira|The Way of All Flesh|
|000Silenced|On the Brink of E...|   Napalm Death|Time Waits For No...|
|000Silenced|On the Brink of E...|   Napalm Death|Time Waits For

In [ ]:
df_listenings.distinct().count()

10605556

In [ ]:
df_listenings=df_listenings.na.drop()

In [ ]:
number_of_rows=df_listenings.count()
number_of_columns=len(df_listenings.columns)

In [ ]:
print(number_of_rows)
print(number_of_columns)

13758905
4


In [ ]:
14650594-13758905

891689

In [ ]:
df_listenings_agg=df_listenings.select('user_id','track').groupby('user_id','track').agg(count("*").alias("count")).orderBy('user_id')

In [ ]:
df_listenings_agg.show()

+-------+--------------------+-----+
|user_id|               track|count|
+-------+--------------------+-----+
| --Seph|Chelsea Hotel - L...|    1|
| --Seph|          Paris 2004|    7|
| --Seph|Airplanes [feat H...|    1|
| --Seph|              Monday|    1|
| --Seph|         The Embrace|    1|
| --Seph|Hungarian Dance No 5|    1|
| --Seph|       Life On Mars?|    1|
| --Seph|Belina (Original ...|    1|
| --Seph| White Winter Hymnal|    3|
| --Seph|               Leloo|    1|
| --Seph|      Hour for magic|    2|
| --Seph|Hungarian Rhapsod...|    1|
| --Seph|     The Way We Were|    1|
| --Seph| Air on the G String|    1|
| --Seph|Vestido Estampado...|    1|
| --Seph|        Window Blues|    1|
| --Seph|  California Waiting|    1|
| --Seph|       Phantom Pt II|    1|
| --Seph|Virus (Luke Fair ...|    1|
| --Seph|   Summa for Strings|    1|
+-------+--------------------+-----+
only showing top 20 rows



In [ ]:
df_listenings_agg=df_listenings_agg.limit(20000)

In [ ]:
df_listenings_agg.show()

+-------+--------------------+-----+
|user_id|               track|count|
+-------+--------------------+-----+
| --Seph|               Leloo|    1|
| --Seph|Virus (Luke Fair ...|    1|
| --Seph|Airplanes [feat H...|    1|
| --Seph|Belina (Original ...|    1|
| --Seph|              Monday|    1|
| --Seph|Hungarian Dance No 5|    1|
| --Seph|       Life On Mars?|    1|
| --Seph|  California Waiting|    1|
| --Seph|       Phantom Pt II|    1|
| --Seph|   Summa for Strings|    1|
| --Seph|      Hour for magic|    2|
| --Seph|Hungarian Rhapsod...|    1|
| --Seph|     The Way We Were|    1|
| --Seph| Air on the G String|    1|
| --Seph|Vestido Estampado...|    1|
| --Seph|        Window Blues|    1|
| --Seph| White Winter Hymnal|    3|
| --Seph|         The Embrace|    1|
| --Seph|          Paris 2004|    7|
| --Seph|Chelsea Hotel - L...|    1|
+-------+--------------------+-----+
only showing top 20 rows



In [ ]:
indexer=[StringIndexer(inputCol=col,outputCol=col+"_index").setHandleInvalid("skip").fit(df_listenings_agg) for col in list(set(df_listenings_agg.columns)-set(["count"]))]

In [ ]:
pipeline=Pipeline(stages=indexer)

In [ ]:
data=pipeline.fit(df_listenings_agg).transform(df_listenings_agg)

In [ ]:
data.show()

+-------+--------------------+-----+-----------+-------------+
|user_id|               track|count|track_index|user_id_index|
+-------+--------------------+-----+-----------+-------------+
| --Seph|          Nightmares|    1|    10600.0|         69.0|
| --Seph|Virus (Luke Fair ...|    1|    15893.0|         69.0|
| --Seph|Airplanes [feat H...|    1|      521.0|         69.0|
| --Seph|Belina (Original ...|    1|     3280.0|         69.0|
| --Seph|              Monday|    1|      334.0|         69.0|
| --Seph|Hungarian Dance No 5|    1|     7555.0|         69.0|
| --Seph|       Life On Mars?|    1|     1164.0|         69.0|
| --Seph|  California Waiting|    1|      195.0|         69.0|
| --Seph|       Phantom Pt II|    1|     1378.0|         69.0|
| --Seph|   Summa for Strings|    1|    13737.0|         69.0|
| --Seph|      Hour for magic|    2|     7492.0|         69.0|
| --Seph|Hungarian Rhapsod...|    1|     7556.0|         69.0|
| --Seph|     The Way We Were|    1|    14958.0|       

In [ ]:
data=data.select("user_id_index","track_index","count").orderBy("user_id_index")

In [ ]:
data.show()

+-------------+-----------+-----+
|user_id_index|track_index|count|
+-------------+-----------+-----+
|          0.0|    10628.0|    1|
|          0.0|     3338.0|    1|
|          0.0|    12168.0|    1|
|          0.0|    11626.0|    2|
|          0.0|    10094.0|    4|
|          0.0|      427.0|    1|
|          0.0|    16878.0|    1|
|          0.0|    11722.0|    1|
|          0.0|    15074.0|    1|
|          0.0|     1359.0|    1|
|          0.0|     5874.0|    1|
|          0.0|    11184.0|    1|
|          0.0|     2372.0|    2|
|          0.0|    14316.0|    1|
|          0.0|     5346.0|    1|
|          0.0|    11194.0|    1|
|          0.0|     2241.0|    1|
|          0.0|     2864.0|    1|
|          0.0|     2663.0|    4|
|          0.0|     6064.0|    1|
+-------------+-----------+-----+
only showing top 20 rows



In [ ]:
(training,test)=data.randomSplit([0.5,0.5])

In [ ]:
USERID="user_id_index"
TRACK="track_index"
COUNT="count"

In [ ]:
als=ALS(maxIter=5,regParam=0.01,userCol=USERID,itemCol=TRACK,ratingCol=COUNT)

In [ ]:
model=als.fit(training)

In [ ]:
predictions=model.transform(test)

In [ ]:
recommendations=model.recommendForAllUsers(10)

/usr/local/lib/python3.7/dist-packages/pyspark/sql/context.py:127: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning


In [ ]:
recommendations.show()

+-------------+--------------------+
|user_id_index|     recommendations|
+-------------+--------------------+
|            0|[{16908, 5.559372...|
|            1|[{349, 10.30236},...|
|            2|[{4460, 13.823327...|
|            3|[{570, 9.78307}, ...|
|            4|[{4460, 8.308568}...|
|            5|[{4460, 9.2230425...|
|            6|[{4460, 9.087256}...|
|            7|[{16968, 9.971637...|
|            8|[{101, 12.077073}...|
|            9|[{0, 11.986891}, ...|
|           10|[{14299, 8.863716...|
|           11|[{14299, 8.730628...|
|           12|[{3345, 7.7782593...|
|           13|[{16908, 9.012111...|
|           14|[{4460, 13.051082...|
|           15|[{16908, 10.13164...|
|           16|[{2380, 9.547393}...|
|           17|[{349, 6.049583},...|
|           18|[{16968, 10.41306...|
|           19|[{16908, 11.28179...|
+-------------+--------------------+
only showing top 20 rows



In [ ]:
recommendations.take(1)

[Row(user_id_index=0, recommendations=[Row(track_index=16908, rating=5.559372425079346), Row(track_index=11753, rating=5.326827049255371), Row(track_index=9099, rating=5.288575172424316), Row(track_index=5041, rating=4.9791436195373535), Row(track_index=165, rating=4.843062877655029), Row(track_index=4460, rating=4.674930095672607), Row(track_index=308, rating=4.263699531555176), Row(track_index=9498, rating=4.2621846199035645), Row(track_index=3345, rating=4.191688060760498), Row(track_index=1461, rating=3.9833147525787354)])]